# Memory Using SQLite

Hướng dẫn này giải thích lớp `SQLChatMessageHistory`, cho phép lưu trữ lịch sử trò chuyện trong bất kỳ cơ sở dữ liệu nào được hỗ trợ bởi `SQLAlchemy`.

`Structured Query Language (SQL)` là một ngôn ngữ dành riêng cho miền (domain-specific language) được sử dụng trong lập trình và được thiết kế để quản lý dữ liệu được lưu trữ trong Hệ thống quản lý cơ sở dữ liệu quan hệ (Relational Database Management System - RDBMS), hoặc để xử lý luồng trong Hệ thống quản lý luồng dữ liệu quan hệ (Relational Data Stream Management System - RDSMS). Nó đặc biệt hữu ích để xử lý dữ liệu có cấu trúc, bao gồm các mối quan hệ giữa các thực thể và biến.

`SQLAlchemy` là một bộ công cụ **SQL** mã nguồn mở và Trình ánh xạ quan hệ đối tượng (Object-Relational Mapper - ORM) cho ngôn ngữ lập trình Python, được phát hành theo Giấy phép MIT.

Để sử dụng cơ sở dữ liệu khác `SQLite`, vui lòng đảm bảo cài đặt trình điều khiển cơ sở dữ liệu thích hợp trước.


In [1]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

load_dotenv(override=True, dotenv_path="../.env")

True

## Usage

Để sử dụng storage, bạn chỉ cần cung cấp 2 thứ sau:

1.  `session_id` - Một định danh duy nhất cho session, ví dụ như tên người dùng, email, chat ID, v.v.

2.  `connection` - Một chuỗi xác định kết nối database. Chuỗi này sẽ được truyền cho hàm `create_engine` của SQLAlchemy.


In [2]:
from langchain_community.chat_message_histories import SQLChatMessageHistory

# Initialize chat history with session ID and database connection.
chat_message_history = SQLChatMessageHistory(
    session_id="sql_history", connection="sqlite:///sqlite.db"
)

In [3]:
# Add a user message
chat_message_history.add_user_message(
    "Hello, nice to meet you! My name is Heesun :) I'm a LangChain developer. I look forward to working with you!"
)
# Add an AI message
chat_message_history.add_ai_message(
    "Hi, Heesun! Nice to meet you. I look forward to working with you too!"
)

In [4]:
chat_message_history.messages

[HumanMessage(content="Hello, nice to meet you! My name is Heesun :) I'm a LangChain developer. I look forward to working with you!", additional_kwargs={}, response_metadata={}),
 AIMessage(content='Hi, Heesun! Nice to meet you. I look forward to working with you too!', additional_kwargs={}, response_metadata={})]

In [5]:
# Clear the session memory
chat_message_history.clear()
chat_message_history.messages

[]

### Adding Metadata

**Metadata** có thể được thêm vào bằng cách tạo trực tiếp các đối tượng `HumanMessage` và `AIMessage`. Phương pháp này cho phép xử lý và ghi log dữ liệu linh hoạt.

**Tham số**:

-   `additional_kwargs` - Lưu trữ các tag hoặc metadata tùy chỉnh, ví dụ như độ ưu tiên hoặc loại task.

-   `response_metadata` - Ghi lại chi tiết phản hồi của AI, bao gồm model, timestamp và số lượng token.

Các trường này tăng cường khả năng debug và theo dõi task thông qua việc lưu trữ dữ liệu chi tiết.


In [6]:
from langchain_core.messages import HumanMessage

# Add a user message with additional metadata.
user_message = HumanMessage(
    content="Can you help me summarize this text?",
    additional_kwargs={"task": "summarization"},
)

# Add the message to chat history.
chat_message_history.add_message(user_message)

In [7]:
chat_message_history.messages

[HumanMessage(content='Can you help me summarize this text?', additional_kwargs={'task': 'summarization'}, response_metadata={})]

In [8]:
from langchain_core.messages import AIMessage

# Add an AI message with response metadata.
ai_message = AIMessage(
    content="Sure! Here's the summary of the provided text.",
    response_metadata={"model": "gpt-4", "token_count": 30, "response_time": "150ms"},
)

# Add the message to chat history.
chat_message_history.add_message(ai_message)

In [9]:
chat_message_history.messages

[HumanMessage(content='Can you help me summarize this text?', additional_kwargs={'task': 'summarization'}, response_metadata={}),
 AIMessage(content="Sure! Here's the summary of the provided text.", additional_kwargs={}, response_metadata={'model': 'gpt-4', 'token_count': 30, 'response_time': '150ms'})]

## Chaining

You can easily integrate this chat history class with [LCEL Runnables](https://wikidocs.net/235884).

In [10]:
from langchain_core.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
)
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.output_parsers import StrOutputParser

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.0)
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        # Placeholder for chat history
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)

# Chaining
chain = prompt | llm | StrOutputParser()

In [11]:
def get_chat_history(user_id, conversation_id):
    return SQLChatMessageHistory(
        table_name=user_id,
        session_id=conversation_id,
        connection="sqlite:///sqlite.db",
    )

Set `config_fields` to provide reference information when retrieving conversation details.

In [12]:
from langchain_core.runnables.utils import ConfigurableFieldSpec

config_fields = [
    ConfigurableFieldSpec(
        id="user_id",
        annotation=str,
        name="User ID",
        description="Unique identifier for a user.",
        default="",
        is_shared=True,
    ),
    ConfigurableFieldSpec(
        id="conversation_id",
        annotation=str,
        name="Conversation ID",
        description="Unique identifier for a conversation.",
        default="",
        is_shared=True,
    ),
]

In [13]:
chain_with_history = RunnableWithMessageHistory(
    chain,
    get_chat_history,
    input_messages_key="question",
    history_messages_key="chat_history",
    # Set parameters for retrieving chat history
    history_factory_config=config_fields,
)

Set the `"user_id"` and `"conversation_id"` key-value pairs under the `"configurable"` key.

In [14]:
# Config settings
config = {"configurable": {"user_id": "user1", "conversation_id": "conversation1"}}

In [15]:
# Execute by passing the question and config
chain_with_history.invoke(
    {"question": "Hi, nice to meet you. My name is Heesun."}, config
)

"Hello Heesun, it's nice to meet you too! How can I assist you today?"

In [16]:
# Execute a follow-up question
chain_with_history.invoke({"question": "What is my name?"}, config)

'Your name is Heesun.'

This time, set the same `user_id` but use a different value for `conversation_id`.

In [17]:
# Config settings
config = {"configurable": {"user_id": "user1", "conversation_id": "conversation2"}}

# Execute by passing the question and config
chain_with_history.invoke({"question": "What is my name?"}, config)

"I'm sorry, but I don't have access to your personal information, including your name. How can I assist you today?"